 # 웹크롤링과 텍스트데이터 시각화

### Outline

### 7. 텍스트데이터 시각화 (B. 네트워크 시각화)
    7B.1 텍스트데이터 웹크롤링
    7B.2 네크워크 시각화

#### 7B.1 텍스트데이터 웹크롤링

- 필요한 라이브러리, 패키지, 모듈 설치

In [ ]:
import re
import requests as req
from bs4 import BeautifulSoup as bs
import time
import math

- 국내학술논문 데이터베이스에서 '빅데이터'를 키워드로 하는 논문 검색

In [ ]:
def keyword_crawler(searchword):

    # URL 및 데이터 지정
    url  = 'http://kiss.kstudy.com/search/sch-result.asp'
    data = {'json': '{"prefixQuery":{"inst":null,"publ":null,"exYear":null,"issn":null,"mbcdls":[],"publPldv":null,"ciInfo":{"title":"등재정보","field":"CI_INFO","value":[{"title":"KCI등재","value":"KCI등재"},{"title":"SCOPUS","value":"SCOPUS"}]}},"collectionQuery":{"queries":[{"field":"전체","value":"'+searchword+'","prefix":""}],"reQueries":[]},"dateRange":null,"sort":null,"isForwardMatch":false,"isContainsAttach":false}',
            'startCount': '0',
            'pageScale': '10'}

    # Post 방식으로 서버에 정보 요청 및 정보 받음
    res = req.post(url = url,data = data)

    # HTML 파싱
    soup = bs(res.text,'html.parser')

    # 원하는 정보 (전체 논문 수) 추출
    N = soup.select('div.list-title.is1 > span.count > span.fc-key')[0].text
    N = re.sub(',','',N)
    N = int(N)
    N = math.ceil(N/100) # 올림

    time.sleep(1) # 시간 1초 지연

    output = []
    for i in range(N):

        # 시작 페이지 번호를 변경할 수 있도록 수정
        data = {'json': '{"prefixQuery":{"inst":null,"publ":null,"exYear":null,"issn":null,"mbcdls":[],"publPldv":null,"ciInfo":{"title":"등재정보","field":"CI_INFO","value":[{"title":"KCI등재","value":"KCI등재"},{"title":"SCOPUS","value":"SCOPUS"}]}},"collectionQuery":{"queries":[{"field":"전체","value":"'+searchword+'","prefix":""}],"reQueries":[]},"dateRange":null,"sort":null,"isForwardMatch":false,"isContainsAttach":false}',
                'startCount': str(i*100),
                'pageScale': '100'}

        # Post 방식으로 서버에 정보 요청 및 정보 받음
        res = req.post(url = url,data = data)

        # HTML 파싱
        soup = bs(res.text,'html.parser')

        # 원하는 정보 (전체 논문 수) 추출
        kws = soup.select('div.key-words')

        kws = [kw.select('span > a') for kw in kws]
        kws = [[x.text.strip() for x in kw] for kw in kws]
        kws = [[re.sub("[^가-힣]","",x) for x in kw] for kw in kws]
        kws = [[x for x in kw if len(x)>0] for kw in kws]

        output += kws

        time.sleep(1) # 시간 1초 지연

    return output

In [ ]:
kws = keyword_crawler('빅데이터')

#### 7B.2 네트워크 시각화

- 필요한 라이브러리, 패키지, 모듈 설치

In [ ]:
import networkx as nx
from itertools import combinations
from collections import Counter

(1) 리스트 안의 각 고유한 요소별로 빈도 계산

In [ ]:
# 이중리스트를 리스트로 변환
kws_long = []
for kw in kws:
    kws_long += kw

In [ ]:
# 각 고유한 요소별 빈도 계산
count = Counter(kws_long)
count.items()

(2) 빈도가 5 이하의 낮은 키워드는 불용어 처리

In [ ]:
stopwords = []
for item in count.items():
    if item[1] <= 5:
        stopwords += [item[0]]
stopwords

(3) 전체 키워드에서 불용어를 제외하고 고유한 키워드의 리스트 생성

In [ ]:
kws_uniq = []
for kw in kws_long:
    if kw not in stopwords:
        kws_uniq += [kw]
kws_uniq = set(kws_uniq)
kws_uniq

(4) 각 논문 내의 키워드 중 불용어 제외

In [ ]:
kws_grp = [[x for x in key if not x in stopwords] for key in kws]
kws_grp

(5) 각 논문 내의 키워드 간 모든 조합을 찾아 리스트로 반환

In [ ]:
kws_pair = [list(combinations(key,2)) for key in kws_grp if len(key)>1]
kws_pair

(6) 키워드 간 조합에 대한 이중 리스트를 리스트로 변환

In [ ]:
# 이중리스트를 리스트로 변환
kws_pair_long = []
for kw in kws_pair:
    kws_pair_long += kw

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.font_manager as fm

- 한글 폰트가 설치된 디렉토리 검색

In [ ]:
font_list = fm.findSystemFonts(fontpaths = None, fontext = 'ttf')
font_list[:]

In [ ]:
font_name = fm.FontProperties(fname="malgun.ttf").get_name() # 위에서 찾은 폰트의 링크 넣어줌
#font_name = fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name() # 위에서 찾은 폰트의 링크 넣어줌
plt.rc('font', family=font_name)

plt.title('빅데이터 논문 키워드 네트워크')

G = nx.Graph()

for kw in kws_uniq:
    G.add_node(kw)

count = Counter(kws_pair_long)
for pair in count.items():
    G.add_edge(pair[0][0],pair[0][1],color='gray',weight=pair[1])

pos=nx.spring_layout(G)

edges = G.edges()
colors = [G[u][v]['color'] for u,v in edges]
weights = [G[u][v]['weight'] for u,v in edges]
nx.draw(G,pos,node_size=0.1,edge_color = colors)
nx.draw_networkx_labels(G, pos, font_family=font_name, font_size=5)

plt.savefig("keyword_graph.png", dpi=1000)

plt.show()

In [ ]:
from IPython.display import Image
Image(filename='Network.png', width=700)

In [ ]:
font_name = fm.FontProperties(fname="malgun.ttf").get_name() # 위에서 찾은 폰트의 링크 넣어줌
#font_name = fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name() # 위에서 찾은 폰트의 링크 넣어줌
plt.rc('font', family=font_name)

plt.title('빅데이터 논문 키워드 네트워크')

nx.draw(G,pos,node_size=0.1,edge_color = colors, width=weights)
nx.draw_networkx_labels(G, pos, font_family=font_name, font_size=5)

plt.show()